# 糖尿病データセットにおけるリッジ回帰モデルのパラメータを用いた実験

このノートブックは、糖尿病データセット上でリッジ回帰モデルを訓練するために、さまざまなパラメータを使って実験するためのものです。

In [ ]:
# 実験ディレクトリからの変更
%cd ..

In [ ]:
import azureml.core
from azureml.core import Workspace

In [ ]:
# 保存された設定ファイルからワークスペースを読み込む
ws = Workspace.from_config()

In [ ]:
import os, shutil

# 実験ファイル用のフォルダを作成する
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# 実験フォルダにデータファイルをコピー
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

# 実験フォルダにtrain関数をコピー
shutil.copy('diabetes_regression/training/train.py', os.path.join(training_folder, "train.py"))

In [ ]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "alpha": 0.3
    },
    "evaluation":
    {

    },
    "scoring":
    {
        
    }
}


In [ ]:
%%writefile $training_folder/diabetes_training.py
# ライブラリのインポート
from azureml.core import Run
import json
import os
import pandas as pd
import shutil

from train import split_data, train_model

# 実験の実行コンテキストの取得
run = Run.get_context()

# 糖尿病データセットの読み込み
print("Loading Data...")
train_df = pd.read_csv('diabetes.csv')

data = split_data(train_df)

# テストするパラメータを指定する
with open("parameters.json") as f:
    pars = json.load(f)
    train_args = pars["training"]

# パラメータのログ
for k, v in train_args.items():
    run.log(k, v)

model, metrics = train_model(data, train_args)

# メトリクスのログ
for k, v in metrics.items():
    run.log(k, v)

# パラメータファイルを出力フォルダに保存する
os.makedirs('outputs', exist_ok=True)
shutil.copy('parameters.json', os.path.join('outputs', 'parameters.json'))
    
run.complete()

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# 推定機の作成
estimator = Estimator(source_directory=training_folder,
                      entry_script='diabetes_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn']
                      )

# 実験の作成
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 推定器に基づいて実験を行う
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

In [ ]:
metrics = run.get_metrics()
for k, v in metrics.items():
        print(k, v)

In [ ]:
for file in run.get_file_names():
    print(file)